# Convolutional Neural Network

- https://github.com/yunjey/pytorch-tutorial/blob/master/tutorials/02-intermediate/convolutional_neural_network/main.py#L33-L53

- ここからGPUを使う
- MNISTとCIFAR10
- CNNはブロック単位で処理した方がよいのでSequentialを使う
- Conv2dはKerasと違って出力のユニットサイズも省略できない
- 自分で計算 or モデルの途中結果サイズをprintしてみる

## MNIST

In [2]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

In [3]:
# Hyperparameters 
num_epochs = 50
batch_size = 100
learning_rate = 0.001

- GPUが使えるか使えないか確認
- 使える場合は自動的にGPU上で動かす

In [33]:
use_gpu = torch.cuda.is_available()

In [9]:
# MNIST Dataset (Images and Labels)
train_dataset = dsets.MNIST(root='./data', 
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = dsets.MNIST(root='./data', 
                           train=False, 
                           transform=transforms.ToTensor())

# Dataset Loader (Input Pipline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [29]:
class CNN(nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.fc = nn.Linear(7 * 7 * 32, 10)
        
    def forward(self, x):
#         print('1:', x.size())
        out = self.layer1(x)
#         print('2:', out.size())
        out = self.layer2(out)
#         print('3:', out.size())
        out = out.view(out.size(0), -1)
#         print('4:', out.size())
        out = self.fc(out)
#         print('5:', out.size())
        return out

- 最後のLinearのユニット数がわからなければ途中結果をprintしてみる
- 7 x 7 x 32 = 1568

- GPUモードで動かすには
- モデルとテンソルデータを`cuda()`でGPUに転送する！

In [35]:
model = CNN()
if use_gpu:
    model.cuda()

In [21]:
# テスト
images, labels = iter(train_loader).next()
print(images.size())
outputs = model(Variable(images))

torch.Size([100, 1, 28, 28])
1: torch.Size([100, 1, 28, 28])
2: torch.Size([100, 16, 14, 14])
3: torch.Size([100, 32, 7, 7])
4: torch.Size([100, 1568])
5: torch.Size([100, 10])


In [31]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

- BatchNormalizationを使っている場合はモデルのモードが重要
- `model.train()` で訓練モード
- `model.eval()` で評価モード

In [ ]:
def train(train_loader):
    model.train()
    running_loss = 0
    for batch_idx, (images, labels) in enumerate(train_loader):
        if use_gpu:
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())
        else:
            images = Variable(images)
            labels = Variable(labels)

        optimizer.zero_grad()
        outputs = model(images)

        loss = criterion(outputs, labels)
        running_loss += loss

        loss.backward()
        optimizer.step()

    train_loss = running_loss / len(train_loader)
    
    return train_loss.data[0]


def valid(test_loader):
    model.eval()
    running_loss = 0
    correct = 0
    total = 0
    for batch_idx, (images, labels) in enumerate(test_loader):
        if use_gpu:
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())
        else:
            images = Variable(images)
            labels = Variable(labels)

        outputs = model(images)

        loss = criterion(outputs, labels)
        running_loss += loss

        _, predicted = torch.max(outputs.data, 1)
        correct += (predicted == labels.data).sum()
        total += labels.size(0)

    val_loss = running_loss / len(test_loader)
    val_acc = correct / total
    
    return val_loss.data[0], val_acc


loss_list = []
val_loss_list = []
val_acc_list = []
for epoch in range(num_epochs):
    loss = train(train_loader)
    val_loss, val_acc = valid(test_loader)

    print('epoch %d, loss: %.4f val_loss: %.4f val_acc: %.4f'
          % (epoch, loss, val_loss, val_acc))
    
    # logging
    loss_list.append(loss)
    val_loss_list.append(val_loss)
    val_acc_list.append(val_acc)

# save the trained model
torch.save(model.state_dict(), 'cnn.pkl')